# Emulator Application

This scripts is used to apply the XGBoost to your CESM simulations.  
Please use compute nodes to do this!    
e.g., with NCAR's Cheyenne: `qsub -X -I -l select=1:ncpus=36 -l walltime=03:00:00 -q regular -A <NCAR_project_id>`

In [1]:
%matplotlib inline
import math
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import xesmf as xe
import pickle
import gc
import time
import xgboost

In [2]:
#Get dataframe from ".nc" file
def get_df_from_nc(ds):
    # define the total variables
    vari_ls = ["lat","lon","time",
               "DMS_SRF","H2O2_SRF","H2SO4_SRF","O3_SRF","SO2_SRF","SOAG_SRF",
               'Mass_so4','Mass_bc','Mass_ncl','Mass_dst','Mass_pom','Mass_soa',
               "T", "RELHUM", "SZA"]
    
    # define the 3D variables
    xr_3d_vari_ls = ['T','RELHUM','O3']
    # define the 2D varibales
    xr_2d_vari_ls = ['SOAG_SRF','DMS_SRF','H2SO4_SRF','H2O2_SRF','SO2_SRF',
                  'bc_a1_SRF','bc_a4_SRF',
                  'dst_a1_SRF','dst_a2_SRF',
                  'ncl_a1_SRF','ncl_a2_SRF',
                  'pom_a1_SRF','pom_a4_SRF',
                  'so4_a1_SRF','so4_a2_SRF',
                  'soa_a1_SRF','soa_a2_SRF',
                  'SZA']
    # merge the 3D variables and convert to dataframe
    ds_3d = ds[xr_3d_vari_ls].isel(lev=-1)
    df_3d = ds_3d.to_dataframe().reset_index()
    df_3d_drop = df_3d.drop(["lev"],axis=1)
    print("Shape of df_3d",df_3d_drop.shape)
    
    # merge the 2D variables and convert to dataframe
    ds_2d = ds[xr_2d_vari_ls]
    df_2d = ds_2d.to_dataframe().reset_index()
    print("Shape of df_2d",df_2d.shape)
    
    df_all =pd.merge(df_3d_drop, df_2d,
                how="outer",
                on=["lat","lon","time"])
    
    # get the specific variables
    df_all['Mass_bc']=df_all['bc_a1_SRF']+df_all['bc_a4_SRF']
    df_all['Mass_dst']=df_all['dst_a1_SRF']+df_all['dst_a2_SRF']
    df_all['Mass_ncl']=df_all['ncl_a1_SRF']+df_all['ncl_a2_SRF']
    df_all['Mass_pom']=df_all['pom_a1_SRF']+df_all['pom_a4_SRF']
    df_all['Mass_so4']=df_all['so4_a1_SRF']+df_all['so4_a2_SRF']
    df_all['Mass_soa']=df_all['soa_a1_SRF']+df_all['soa_a2_SRF']
    
    df_all['O3_SRF']=df_all['O3']
    df_all["RELHUM"].where(df_all["RELHUM"]<100, 100, inplace=True)
    
    df = df_all[vari_ls]
    df['SZA'] = df_all['SZA']*math.pi / 180
    df['RELHUM'] = df_all['RELHUM'] / 100 
    print("Shape of df",df.shape)
    
    return df

def get_df_from_regrid_nc(ds):
    # define the total variables
    vari_ls = ["lat","lon","time",
               "DMS_SRF","H2O2_SRF","H2SO4_SRF","O3_SRF","SO2_SRF","SOAG_SRF",
               'Mass_so4','Mass_bc','Mass_ncl','Mass_dst','Mass_pom','Mass_soa',
               "T", "RELHUM", "SZA"]
    
    # define the 3D variables
    xr_3d_vari_ls = ['T','RELHUM','O3']
    # define the 2D varibales
    xr_2d_vari_ls = ['SOAG_SRF','DMS_SRF','H2SO4_SRF','H2O2_SRF','SO2_SRF',
                  'bc_a1_SRF','bc_a4_SRF',
                  'dst_a1_SRF','dst_a2_SRF',
                  'ncl_a1_SRF','ncl_a2_SRF',
                  'pom_a1_SRF','pom_a4_SRF',
                  'so4_a1_SRF','so4_a2_SRF',
                  'soa_a1_SRF','soa_a2_SRF',
                  'SZA']
    # merge the 3D variables and convert to dataframe
    ds_3d = ds[xr_3d_vari_ls]
    df_3d = ds_3d.to_dataframe().reset_index()
    print("Shape of df_3d",df_3d.shape)
    
    # merge the 2D variables and convert to dataframe
    ds_2d = ds[xr_2d_vari_ls]
    df_2d = ds_2d.to_dataframe().reset_index()
    print("Shape of df_2d",df_2d.shape)
    
    df_all =pd.merge(df_3d, df_2d,
                how="outer",
                on=["lat","lon","time"])
    
    # get the specific variables
    df_all['Mass_bc']=df_all['bc_a1_SRF']+df_all['bc_a4_SRF']
    df_all['Mass_dst']=df_all['dst_a1_SRF']+df_all['dst_a2_SRF']
    df_all['Mass_ncl']=df_all['ncl_a1_SRF']+df_all['ncl_a2_SRF']
    df_all['Mass_pom']=df_all['pom_a1_SRF']+df_all['pom_a4_SRF']
    df_all['Mass_so4']=df_all['so4_a1_SRF']+df_all['so4_a2_SRF']
    df_all['Mass_soa']=df_all['soa_a1_SRF']+df_all['soa_a2_SRF']
    
    df_all['O3_SRF']=df_all['O3']
    df_all["RELHUM"].where(df_all["RELHUM"]<100, 100, inplace=True)
    
    df = df_all[vari_ls]
    df['SZA'] = df_all['SZA']*math.pi / 180
    df['RELHUM'] = df_all['RELHUM'] / 100 
    print("Shape of df",df.shape)
    
    return df



# apply the ML and return to netcdf
def predict_chi_ls(df, vari, chi_ls):
    df_copy = df.copy()
    for chi in chi_ls:
        XGBreg_load=pickle.load(open("./xgb_model/"+chi+".dat","rb"))
        print("start to get",chi)
        time_s=time.time()
        df_copy[chi]=XGBreg_load.predict(df[vari])
        print("It took",time.time()-time_s,"to get chi")
        del XGBreg_load
        gc.collect()
    
    time_s=time.time()
    df_xr = df_copy.set_index(["time","lat","lon"]).to_xarray()
    print("It took",time.time()-time_s,"to get dataset")
    
    return df_xr

In [3]:
chi_ls = ['chi_hyg','chi_opt1','chi_opt2','chi_abd']
vari = ["DMS_SRF","H2O2_SRF","H2SO4_SRF","O3_SRF","SO2_SRF","SOAG_SRF",
        'Mass_so4','Mass_bc','Mass_ncl','Mass_dst','Mass_pom','Mass_soa',
        "T", "RELHUM", "SZA"]

## Apply the emulators to CESM simulations

In [4]:
%%time
# define the path of your simulations
path="/glade/scratch/zhonghua/BWs_CESM/branch_1111_F09_2011/"
# define the path for saving your mixing state indices estimates
save_path="/glade/scratch/zhonghua/PartMC-CESM_s/pred/f09/2011_spinup/"
for i in range(1,13):
    month=str(i).zfill(2)
    print("start the month:",month)
    # My CESM simulations is based on different month, January is 01.nc...
    ds=xr.open_dataset(path+month+".nc")
    df=get_df_from_nc(ds)
    print("The shape is:", df.shape)
    df_pred = predict_chi_ls(df, vari, chi_ls)
    df_pred.to_netcdf(save_path+month+".nc")
    
    del ds,df,df_pred
    gc.collect()
    print("\n")

start the month: 01
Shape of df_3d (13713408, 6)
Shape of df_2d (13713408, 21)


/glade/work/zhonghua/personal_clone_name/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/glade/work/zhonghua/personal_clone_name/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Shape of df (13713408, 18)
The shape is: (13713408, 18)
start to get chi_hyg
It took 35.670507192611694 to get chi
start to get chi_opt1
It took 35.229050636291504 to get chi
start to get chi_opt2
It took 42.22355937957764 to get chi
start to get chi_abd
It took 35.096696615219116 to get chi
It took 55.121297121047974 to get dataset


start the month: 02
Shape of df_3d (12386304, 6)
Shape of df_2d (12386304, 21)
Shape of df (12386304, 18)
The shape is: (12386304, 18)
start to get chi_hyg
It took 34.61480402946472 to get chi
start to get chi_opt1
It took 34.392346143722534 to get chi
start to get chi_opt2
It took 42.4433913230896 to get chi
start to get chi_abd
It took 34.39589047431946 to get chi
It took 49.42144298553467 to get dataset


start the month: 03
Shape of df_3d (13713408, 6)
Shape of df_2d (13713408, 21)
Shape of df (13713408, 18)
The shape is: (13713408, 18)
start to get chi_hyg
It took 35.22023868560791 to get chi
start to get chi_opt1
It took 35.45146894454956 to get chi